# Quizzes for Quantum Computing @ TSpark 2022 Quantum+ Camp

## Install Prerequisites

In [ ]:
!pip install tensorcircuit
!pip install qiskit
!pip install optax
!pip install pylatexenc

## Import Libraries

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorcircuit as tc
import sys
import math
import scipy as sp
import qiskit

### Test Script 1.1
Make sure this script can be executed

In [ ]:
K = tc.set_backend("tensorflow")


@K.jit
def exp_z(param):
    c = tc.Circuit(1)
    c.rx(0, theta=param)
    return K.real(c.expectation_ps(z=[0]))


grad_z = K.grad(exp_z)

params = K.convert_to_tensor(np.arange(0, 2 * np.pi, 0.01))

plt.plot(params, [exp_z(param) for param in params], label=r"$\langle Z\rangle$")
plt.plot(
    params,
    [grad_z(param) for param in params],
    label=r"$\frac{\partial \langle Z\rangle}{\partial \theta}$",
)
plt.legend()
plt.xlabel(r"$\theta$")
plt.show()

## TensorCircuit Test Script

In [ ]:
"""
Backend agnostic linear regression with gradient descent optimization:
a demonstration on most of core features and paradigm of tensorcircuit
"""

# this script shows how backend agnostic magic works, no code change is required to switch backend
# we also include jit, vmap and AD features in this pure classical example
# this demonstrates that tensorcircuit can serve as a solid unified ML library without any "quantumness"

sys.path.insert(0, "../")

# (x, y) data preparation

nsamples = 100
k0 = 1.0
b0 = 0.0

xs0 = np.random.uniform(low=-1, high=1, size=[nsamples])
ys0 = k0 * xs0 + b0 + np.random.normal(scale=0.1, size=[nsamples])


def lr(xs, ys):
    """
    fully ML backend agnostic linear regression implementation
    """
    # construct the loss
    def loss_pointwise(x, y, param):
        k, b = param["k"], param["b"]
        yp = k * x + b
        return (yp - y) ** 2

    # we suppose this loss function only works for scalar, so that we can show the usage of ``vmap``

    loss_vmap = tc.backend.vmap(loss_pointwise, vectorized_argnums=(0, 1))

    # now we define the total loss for all data

    def loss(xs, ys, param):
        losses = loss_vmap(xs, ys, param)
        return tc.backend.sum(losses)

    # we get the jitted function to evaluate loss and its derivatives wrt. param

    loss_and_grad = tc.backend.jit(tc.backend.value_and_grad(loss, argnums=2))

    # setup initial values and optimizers

    weight = {"k": tc.backend.implicit_randn(), "b": tc.backend.implicit_randn()}

    if tc.backend.name == "tensorflow":
        import tensorflow as tf

        opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-2))
    elif tc.backend.name == "jax":
        import optax

        opt = tc.backend.optimizer(optax.adam(1e-2))
    else:
        raise ValueError("Unsupported backend")

    # gradient descent optimization loop
    maxstep = 500
    for i in range(maxstep):
        loss, grad = loss_and_grad(xs, ys, weight)
        weight = opt.update(grad, weight)
        if i % 100 == 0 or i == maxstep - 1:
            print("optimized MSE loss after %s round: " % i, tc.backend.numpy(loss))

    return tc.backend.numpy(weight["k"]), tc.backend.numpy(weight["b"])


if __name__ == "__main__":

    for n in ["tensorflow", "jax"]:
        with tc.runtime_backend(n):  # runtime backend switch with context manager
            print("~~~~~~~~ using %s backend ~~~~~~~~" % n)
            xs_tensor, ys_tensor = tc.array_to_tensor(xs0, ys0, dtype="float32")
            print("predicted coefficient", lr(xs_tensor, ys_tensor))

## 1.2 向量和矩阵

### Script 1.2.1 矩阵旋转
向量旋转，将vec旋转theta度（theta为radian）

In [ ]:
theta = float(input())
vec = np.array([1,0])

array_r_theta = np.array([[math.cos(theta),math.sin(theta)],[-1*math.sin(theta),math.cos(theta)]])

result = array_r_theta @ vec

### Script 1.2.2 矩阵指数
证明$\mathrm{e}^{i\theta \hat{P}} = \cos{\theta} I + i \sin{\theta} \hat{P}$ ($\hat P$ equals to $\hat X\mathrm{,\ } \hat Y\mathrm{,\ }  \hat Z$)

In [ ]:
tc.set_backend("tensorflow")

theta = float(input())

X = tc.gates._x_matrix
Y = tc.gates._y_matrix
Z = tc.gates._z_matrix
I = tc.gates._i_matrix

r1X = sp.linalg.expm(theta*1j*X)#e^ i theta P
r1Y = sp.linalg.expm(theta*1j*Y)
r1Z = sp.linalg.expm(theta*1j*Z)

r2X = math.cos(theta) * I + 1j * math.sin(theta) * X#cos theta I + i sin theta P
r2Y = math.cos(theta) * I + 1j * math.sin(theta) * Y
r2Z = math.cos(theta) * I + 1j * math.sin(theta) * Z

print(X)
print(r1X)
print(r2X)
print("\n")
print(Y)
print(r1Y)
print(r2Y)
print("\n")
print(Z)
print(r1Z)
print(r2Z)
print("\n")

### Script 1.2.3 矩阵关于向量的期望
$\hat Q$ 关于 $\vec v$ 的期望

In [ ]:
tc.set_backend("tensorflow")

def assign_pauli(index):#assigning Pauli Matrix
  if(index == 0):
    return tc.gates._x_matrix
  elif(index == 1):
    return tc.gates._y_matrix
  else:
    return tc.gates._z_matrix

mP = assign_pauli(2)#Choosing Pauli Matrix (0-x, 1-y other numbers -z)
mQ = assign_pauli(1)

def get_expt(theta):#get the expection for the vector to spin theta degrees
  v0 = np.array([[1],[0]])
  vp = sp.linalg.expm(theta*1j/2*mP)
  v1 = vp @ v0
  v1_tc = v1.T.conj()
  expt = v1_tc @ mQ @ v1
  return expt

yrcoord = []#real part
yicoord = []#imgainary part

xcoord = np.linspace(0, 2*math.pi, 10000)#create x-coords list, prems: initial point, ending point, and sample number

for i in xcoord:
  y = get_expt(i)[0][0]
  yrcoord.append(y.real)
  yicoord.append(y.imag)

plt.plot(xcoord,yrcoord,xcoord,yicoord)
plt.show()#draw graph

### Script 1.2.4 张量积
计算$\sum^{n-1}_{i=0} Z_i+\sum^{n-2}_{i=0} X_iX_{i+1}$，并计算其在向量（1，0，0，0，0……）的期望值

In [ ]:
X = tc.gates._x_matrix
Z = tc.gates._z_matrix
I = tc.gates._i_matrix

Gn = int(input())

def get_expt(vec, ma):
  vec_tc = vec.T.conj()
  vec_rem = vec_tc @ ma
  return vec_rem @ vec

def get_tp(mat, n, len):
  if(n==0):
    ret = mat
    for index_i in range (1, len):
      ret = np.kron(ret, I)
  else:
    ret = I
    for index_i in range(1, n):
      ret = np.kron(ret, I)
    ret = np.kron(ret, mat)
    for index_i in range(n+1, len):
      ret = np.kron(ret, I)
  return ret

res = np.zeros([2**Gn,2**Gn])
for index_z in range(0,Gn):
  res = res + get_tp(Z, index_z, Gn)

for index_x in range(0, Gn-1):
  res = res + get_tp(X, index_x, Gn) @ get_tp(X, index_x + 1, Gn)

vec_standard = np.zeros([2**Gn])
vec_standard[0] = 1
print(get_expt(vec_standard,res))

### Script 1.2.5 狄拉克符号
使用狄拉克符号（例 $\ket{0}= \begin{pmatrix} 1 \\ 0 \end{pmatrix}$ ）
<!--$\ket{0} \left| 0 \right> \left< 0 | 0 \right> \bra{0}\ket{0} \braket{0|0} \bra{0}\hat{A}\ket{0} \braket{0 | \hat{A} | 0} \begin{pmatrix}1 & 0\\ 0 & 1\end{pmatrix} \begin{matrix}1 & 0\\ 0 & 1\end{matrix} \begin{bmatrix}1 & 0\\ 0 & 1\end{bmatrix} \begin{Bmatrix}1 & 0\\ 0 & 1\end{Bmatrix}$-->

In [ ]:
#Useful link: https://docs.microsoft.com/en-us/azure/quantum/concepts-dirac-notation
dirac_in = input()

ret = np.array([1])
for i in range(0, len(dirac_in)):
  if(dirac_in[i]=="0"):
    ret = np.kron(ret,[[1],[0]])
  else:
    ret = np.kron(ret,[[0],[1]])

print(ret)

## 1.3 导数和梯度下降

### Script 1.3.3 单比特位移

In [ ]:
K = tc.set_backend("tensorflow")

def assign_pauli(index):
  if(index == 0):
    return tc.gates._x_matrix
  elif(index == 1):
    return tc.gates._y_matrix
  else:
    return tc.gates._z_matrix

p1331 = assign_pauli(1)
p1332 = assign_pauli(2)

def function_f_133(theta):
  theta = K.cast(theta, dtype=tc.dtypestr)
  ret = tc.array_to_tensor(np.array([[1.,0]]))
  ret = ret@ K.expm(theta*(-1j/2)*tc.array_to_tensor(p1331))
  ret = ret@ tc.array_to_tensor(p1332) @ K.expm(theta*1.j/2*tc.array_to_tensor(p1331))
  ret = ret@ tc.array_to_tensor(np.array([[1.],[0]]))
  return ret


ycord = []

xcord = np.linspace(-10, 10, 100)

for i in xcord:
  y = function_f_133(i)[0][0]
  ycord.append(y)

plt.plot(xcord,ycord)
plt.show()

### 1.3.4 梯度下降（利用.grad和两个单数据变量计算梯度下降）

In [ ]:
lr = 0.7

x = K.convert_to_tensor(2.0)
y = K.convert_to_tensor(2.0)

def fff(nx,ny):
  return nx**2+ny**2

gf = K.grad(fff, argnums=(0,1))
for i in range(0,100):
  slc = gf(x,y)
  x = x - slc[0]*lr
  y = y - slc[1]*lr
  print(x)
  print(y)


### 1.3.4 梯度下降（利用.grad和一个张量变量计算梯度下降）

In [ ]:
lr = 0.7

q = tc.array_to_tensor([2.0,3.0])

def ffff(q):
  return q[0]**2+q[1]**2

gff = K.grad(ffff)
for i in range(0,100):
  slc = gff(q)
  print(q[0])
  q -= slc * lr
  print(q)

## 1.4 量子线路

### Script 1.4.1 态生成

#### Script 1.4.1.1

In [ ]:
n=6

c=tc.Circuit(n)

c.h(0)

for i in range(1,n):
    c.cnot(0,i)

c.draw(output="mpl")

#### Script 1.4.1.2

In [ ]:
n=6

c=tc.Circuit(n)

for i in range(n):
    c.h(i)

c.draw(output="mpl")

#### Script 1.4.1.3

In [ ]:
c=tc.Circuit(4)

u1=np.array([[1,0],[0,1]])
u2=np.array([[1,0],[0,1]])
z=np.array([[1,0],[0,-1]])

c.h(0)
c.h(1)
c.multicontrol(*range(2),ctrl=[1],unitary=z)

c.cnot(0,2)
c.cnot(1,3)

c.multicontrol(2,0,ctrl=[0],unitary=u1)

c.multicontrol(3,1,ctrl=[1],unitary=z)

c.multicontrol(3,1,ctrl=[1],unitary=u2)

while c.measure(1)==1:
    continue
c.draw(output="mpl")

### Script 1.4.2 多比特下的参数平移

In [ ]:
K = tc.set_backend("tensorflow")
def exp_sumz(n, nlayers, param):
    c = tc.Circuit(n)
    for j in range(nlayers):
        for i in range(n):
            c.rx(i, theta=param[i, j])
        for i in range(n - 1):
            c.cnot(i, i + 1)
    c.draw(output="mpl")
    return K.real(K.sum([c.expectation_ps(z=[i]) for i in range(n)]))

n, nlayers = 3, 3
param = K.randn([n, nlayers],dtype=tf.complex64)


for dx in range(n):
  for dy in range(nlayers):
    np.arange(0,10,0.1)
    deltas = tc.array_to_tensor(np.arange(0, 10, 0.1))
    results = []
    for i in range(deltas.shape[0]) :
      Delta=np.zeros([n, nlayers])
      Delta[dx][dy] = deltas[i]
      results.append(exp_sumz(n, nlayers, param + tc.array_to_tensor(Delta)))
    plt.plot(deltas, results)
    plt.xlabel("({},{})".format(dx,dy))
    plt.show()

### Script 1.4.3 量子门分解

In [ ]:
val = 2.0
c = tc.Circuit(2)
c.RZ(1, theta = val)
c.z(1)
print(c.matrix())
for i in range(4):
  print(c.matrix()[i][i])
c.draw(output="mpl")

## 1.5 量子算法

### Script 1.5.1 oracle

In [ ]:
c=tc.Circuit(14)

c.x(0)
c.x(2)
c.x(5)

#input

a=np.array([[0 for i in range(6)] for j in range(6)])

a[0,1]=6
a[1,2]=7
a[0,3]=8
a[1,4]=9
a[2,5]=10
a[3,4]=11
a[4,5]=12

def pd(i,j):
    c.cnot(i,a[i,j])
    c.cnot(j,a[i,j])
for i in range(6):
    for j in range(6):
        if a[i,j]!=0:
            pd(i,j)
            
u=np.array([[0,1],[1,0]])

c.multicontrol(*range(6,14),ctrl=[1 for i in range(7)],unitary=u)

for i in range(5,-1,-1):
    for j in range(5,-1,-1):
        if a[i,j]!=0:
            pd(i,j)   

print(" ")
c.draw(output="mpl")

### Script 1.5.2 Grover

In [ ]:
c=tc.Circuit(7)
n=6
m=2
h1=np.array([[1,1],[1,-1]])/math.sqrt(2)
hn=h1
thi=math.asin(math.sqrt(m/n))/2

def CI(x):#Nearest integer
    y=int(x)
    if x-y>=0.5:
        return y+1
    else:
        return y

R=CI(math.acos(math.sqrt(m/n))/thi)#Number of cycles

def hnn():#n H-gates
    for i in range(n):
        c.h(i)
    
u=np.array([[0,1],[1,0]])#x-gate, optimize needed

def psg():#phase shift gate
    c.multicontrol(*range(7),ctrl=[0 for i in range(6)],unitary=u)
    
c.h(6)
c.z(6)


def oracle():#oracle
    c.cnot(0,3)
    c.cnot(1,4)
    c.cnot(2,5)
    c.cnot(1,2)
    c.cnot(0,1)
                
    c.multicontrol(*range(1,7),ctrl=[1 for i in range(5)],unitary=u)
                
    c.cnot(0,1)
    c.cnot(1,2)
    c.cnot(2,5)
    c.cnot(1,4)
    c.cnot(0,3)

def grover():#grover
    oracle()
    hnn()
    psg()
    hnn()

#OUTPUT
    
hnn()

for i in range(R):
    grover()

c.draw(output="mpl")